# Import

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import psycopg2
import pandas as pd
import time
import math
import datetime

In [50]:
#sns.set()

In [2]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()

In [52]:
start_time = time.time()
cur.execute("SELECT DISTINCT numr_cart FROM stif_validation_unitaire")
numr_cart=cur.fetchall()
len(numr_cart)
print("--- %s seconds ---" % (time.time() - start_time))

--- 7.406815528869629 seconds ---


In [53]:
start_time = time.time()
cur.execute("SELECT numr_cart FROM stif_validation_unitaire")
cur.rowcount()
print("--- %s seconds ---" % (time.time() - start_time))

TypeError: 'int' object is not callable

# Validations times each day

In [ ]:
cur.execute('''SELECT COUNT(*) AS count
            ,EXTRACT(isodow FROM dat_vald) AS day
            FROM stif_validation_unitaire
            GROUP BY day''')
cur.fetchall()

# Validation distribution

In [ ]:
cur.execute('''SELECT heur_vald,COUNT(*) AS count
            FROM stif_validation_unitaire
            GROUP BY 1
            ''')
num_vald=cur.fetchall()
num_vald=pd.DataFrame(num_vald)

In [ ]:
num_vald[1]

In [ ]:
plt.plot(num_vald[0],num_vald[1])
plt.axvline(x='06:00:00',color='g',linestyle='--')
plt.axvline(x='10:00:00',color='g',linestyle='--')
plt.axvline(x='15:00:00',color='g',linestyle='--')
plt.axvline(x='21:00:00',color='g',linestyle='--')
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Time intervals according to distribution of validations')
plt.savefig('time_interval.png',format='png',dpi=1200)
plt.show()

# Validation times per hour

In [ ]:
cur.execute('''SELECT COUNT(*) AS count
            ,EXTRACT(hour FROM heur_vald) AS hour
            ,EXTRACT(isodow FROM dat_vald) AS day
            FROM stif_validation_unitaire
            GROUP BY hour,day
            ''')
num_vald_hour=cur.fetchall()
num_vald_hour=pd.DataFrame(num_vald_hour)

In [ ]:
num_vald_hour.columns=['count','hour','day']

In [ ]:
num_vald_hour.to_csv('num_vald_hour.csv')

In [ ]:
num_vald_hour=pd.read_csv('num_vald_hour.csv')

In [ ]:
num_vald_hour

In [ ]:
num_vald_hour.loc[num_vald_hour['day'].isin((1,2,3,4,5))].groupby('hour').mean()

In [ ]:
num_vald_hour_weekday=num_vald_hour.loc[num_vald_hour['day'].isin((1,2,3,4,5))].groupby('hour').mean()

In [ ]:
plt.bar(range(0,24),num_vald_hour_weekday['count'])
plt.xlabel('Hours of day')
plt.ylabel('Validation times')
plt.ylim(ymin=0,ymax=400000)
plt.title('Number of validations per hour during weekday')
plt.show()

In [ ]:
num_vald_hour_weekend=num_vald_hour.loc[num_vald_hour['day'].isin((6,7))].groupby('hour').mean()

In [ ]:
plt.bar(range(0,24),num_vald_hour_weekend['count'])
plt.xlabel('Hours of day')
plt.ylabel('Validation times')
plt.ylim(ymin=0,ymax=400000)
plt.title('Number of validations per hour during weekend')
plt.show()

# Validation times each day

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,COUNT(heur_vald) AS count
            FROM stif_validation_unitaire
            GROUP BY 1,2''')
vald_each_day=cur.fetchall()
vald_each_day=pd.DataFrame(vald_each_day)
vald_each_day.columns=['numr_cart','dat_vald','count']
vald_each_day.to_csv('vald_each_day.csv')

In [ ]:
vald_each_day

In [ ]:
vald_each_day[2].describe()

In [ ]:
vald_each_day=pd.read_csv('vald_each_day.csv')

In [ ]:
vald_each_day

In [ ]:
sns.distplot(vald_each_day['count'])
plt.xlabel('validations')
plt.title('Distribution of valdation times each day')

In [ ]:
vald_each_day['count'].quantile(0.999)

In [ ]:
sns.distplot(vald_each_day.loc[vald_each_day['count']>20]['count'])
plt.xlabel('validations')
plt.title('Distribution of valdation times each day')

In [ ]:
sns.boxplot(vald_each_day[2])
plt.xlabel('validations')
plt.title('Validation times per day')

In [ ]:
cur.execute('''SELECT numr_cart,EXTRACT( isodow FROM dat_vald),COUNT(heur_vald) AS count
            FROM stif_validation_unitaire
            GROUP BY 1,2''')
vald_each_weekday=cur.fetchall()
vald_each_weekday=pd.DataFrame(vald_each_weekday)
vald_each_weekday.columns=['numr_cart','weekday','count']
vald_each_weekday.to_csv('vald_each_weekday.csv')

In [ ]:
vald_each_weekday.groupby('weekday').describe()

In [ ]:
sns.boxplot(x='weekday',y='count',data=vald_each_weekday)
plt.title('Validation times each day of week')

# Validation times each station

In [ ]:
cur.execute("""SELECT a.cod_tlb_arrt_sttn, COUNT(a.cod_tlb_arrt_sttn) as count, b.libl_long_arrt_sttn, b.mod_bus, b.mod_rer
            FROM stif_validation_unitaire a, stif_arret_station_physique b
            WHERE b.cod_tlb_arrt_sttn = a.cod_tlb_arrt_sttn
            GROUP BY 1,3,4,5
            ORDER BY count desc
            """)
freq_station=cur.fetchall()
freq_stat=pd.DataFrame(freq_station)
freq_stat.columns = ['cod_tlb_arrt_sttn','count','libl_long_arrt_sttn','mod_bus','mod_rer']

In [ ]:
freq_stat

In [ ]:
freq_stat.to_csv('freq_station.csv')

In [ ]:
freq_station=pd.read_csv('freq_station.csv')

In [ ]:
freq_station[2:12]

In [ ]:
sns.barplot(x='count',y='libl_long_arrt_sttn',data=freq_station[2:12])
plt.title('Top 10 most frequent station')

In [ ]:
cur.execute('''SELECT *
            FROM stif_arret_station_physique
            WHERE libl_long_arrt_sttn = 'MONTPARNASSE'
            ''')
cur.fetchall()

There are some stations which have reocrds with same name but different code, it's mainly because of La structure de l’objet « arrêt », so for these stations we merge those different records by adding up their times and counting for one single record.

# Validation times each passenger

In [ ]:
cur.execute('''SELECT numr_cart, COUNT(numr_cart) AS count
            FROM stif_validation_unitaire
            GROUP BY numr_cart
            ORDER BY count desc
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,w.cod_tlb_arrt_sttn,c.libl_long_arrt_sttn
            FROM stif_validation_unitaire w, stif_arret_station_physique c
            WHERE numr_cart = 1755513924669118709
            AND c.cod_tlb_arrt_sttn = w.cod_tlb_arrt_sttn
            ORDER BY (dat_vald,heur_vald)
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,w.cod_tlb_arrt_sttn,c.libl_long_arrt_sttn
            FROM stif_validation_unitaire w, stif_arret_station_physique c
            WHERE numr_cart = 1588994139548062425
            AND c.cod_tlb_arrt_sttn = w.cod_tlb_arrt_sttn
            ORDER BY (dat_vald,heur_vald)
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart, COUNT(numr_cart) AS count
            FROM stif_validation_unitaire
            GROUP BY numr_cart
            ORDER BY count 
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart
            FROM stif_validation_unitaire
            GROUP BY numr_cart
            HAVING COUNT(numr_cart) < 4000 AND COUNT(numr_cart) > 5''')
numr_cart=cur.fetchall()
len(numr_cart)

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,w.cod_tlb_arrt_sttn,c.libl_long_arrt_sttn
            FROM stif_validation_unitaire w, stif_arret_station_physique c
            WHERE numr_cart = 11886830948974521
            AND c.cod_tlb_arrt_sttn = w.cod_tlb_arrt_sttn
            ORDER BY (dat_vald,heur_vald)
            ''')
cur.rowcount

# Validation times of passengers in different cart type

In [ ]:
cur.execute('''SELECT idnt_titr_clnt, COUNT(idnt_titr_clnt)
            FROM stif_contrat_carte
            GROUP BY idnt_titr_clnt
            ORDER BY count desc
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,idnt_titr_refr
            FROM stif_contrat_carte
            WHERE idnt_titr_refr = '3'
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,w.cod_tlb_arrt_sttn,c.libl_long_arrt_sttn
            FROM stif_validation_unitaire w, stif_arret_station_physique c
            WHERE numr_cart = 72246404613679100956
            AND c.cod_tlb_arrt_sttn = w.cod_tlb_arrt_sttn
            ORDER BY (dat_vald,heur_vald)
            ''')
cur.fetchall()

In [ ]:
 cur.execute('''SELECT AVG(count)
             FROM(
             SELECT numr_cart,dat_vald,COUNT(numr_cart) AS count
             FROM stif_validation_unitaire
             GROUP BY 1,2
             ) AS derivedTable
             ''')
cur.fetchall()

# Validation times each day of week

In [ ]:
 cur.execute('''SELECT day,AVG(count)
             FROM(
             SELECT numr_cart,dat_vald,EXTRACT(isodow FROM dat_vald) AS day,COUNT(numr_cart) AS count
             FROM stif_validation_unitaire
             GROUP BY 1,2
             ) AS derivedTable
             GROUP BY 1''')
avg_day=cur.fetchall()

In [ ]:
avg_days=[None]*7
for i in range(1,8):
    avg_days[i-1]=avg_day[i-1][1]

In [ ]:
plt.bar(range(1,8),avg_days)
plt.show()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,min(heur_vald)
            FROM stif_validation_unitaire
            WHERE numr_cart = '1027094693899596069'
            GROUP BY 1,2''')
cur.fetchall()

# P

In [ ]:
cur.execute('''SELECT (cod_tlb_arrt_sttn,libl_long_arrt_sttn,mod_rer) 
            FROM stif_arret_station_physique 
            WHERE libl_long_arrt_sttn ~ 'MASSY-PALAISEAU'
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,min(heur_vald)
            FROM stif_validation_unitaire
            WHERE cod_tlb_arrt_sttn in ('0030050140400','0030050180602')
            AND date_part('hour',heur_vald) = 8
            GROUP BY 1''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,w.cod_tlb_arrt_sttn,c.libl_long_arrt_sttn
            FROM stif_validation_unitaire w, stif_arret_station_physique c
            WHERE numr_cart = 38406801252407928
            AND c.cod_tlb_arrt_sttn = w.cod_tlb_arrt_sttn
            ORDER BY (dat_vald,heur_vald)
            ''')
cur.fetchall()

# Standard deviation of the first validation of each person

In [ ]:
cur.execute('''SELECT numr_cart,stddev(second) AS sd
            FROM(
            SELECT numr_cart, dat_vald, EXTRACT(EPOCH FROM heur_vald) AS second
            FROM stif_validation_unitaire
            GROUP BY 1,2,heur_vald
            ) AS derivedTable
            GROUP BY 1
            ORDER BY sd ''')
cur.fetchall()

In [ ]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()

In [ ]:
cur.execute('''SELECT numr_cart,EXTRACT(isodow FROM dat_vald),stddev(60*hour+minute) AS sd
            FROM(
            SELECT numr_cart, dat_vald, EXTRACT (HOUR FROM heur_vald) AS hour,EXTRACT(MINUTE FROM heur_vald) AS minute
            FROM stif_validation_unitaire
            GROUP BY 1,2,heur_vald
            ) AS derivedTable
            GROUP BY 1,2
            ORDER BY sd ''')
sd_day=cur.fetchall()
sd_day=pd.DataFrame(sd_day)
sd_day=sd_day.dropna(how='any')

In [ ]:
sd_day.columns=['numr_cart','day','sd']

In [ ]:
sd_day

In [ ]:
sd_day.to_csv('sd_day.csv')

In [ ]:
sd_day=pd.read_csv('sd_day.csv')

In [ ]:
sd_day.groupby('day').describe()

In [ ]:
sd=sd_day

In [ ]:
plt.boxplot([sd.loc[sd[1] == 1][2],sd.loc[sd[1] == 2][2],sd.loc[sd[1] == 3][2],sd.loc[sd[1] == 4][2],sd.loc[sd[1] == 5][2],sd.loc[sd[1] == 6][2],sd.loc[sd[1] == 7][2]])
plt.title('Boxplot of standard deviation of each first validation per day for every person')
plt.ylabel('SD(min)')
weekday_ticks=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
plt.xticks(range(1,8),weekday_ticks)
plt.show()

# Number of unique stations(zdl) during weekday and weekend

In [ ]:
cur.execute('''SELECT w.numr_cart,count,countw
            FROM(SELECT numr_cart,dat_vald,COUNT(DISTINCT cod_tlb_arrt_sttn) AS count
            FROM stif_validation_unitaire
            WHERE EXTRACT(isodow FROM dat_vald) IN (1,2,3,4,5)
            GROUP BY 1,2
            ) AS w,
            (SELECT numr_cart,dat_vald,COUNT(DISTINCT cod_tlb_arrt_sttn) AS countw
            FROM stif_validation_unitaire
            WHERE EXTRACT(isodow FROM dat_vald) IN (6,7)
            GROUP BY 1,2
            ) AS c
            WHERE w.numr_cart = c.numr_cart
            ORDER BY count desc
            ''')
num_uniq_stat=cur.fetchall()

In [ ]:
num_uniq_stat=pd.DataFrame(num_uniq_stat)
num_uniq_stat.columns=['numr_cart','weekday','weekend']
num_uniq_stat

In [ ]:
cur.execute('''SELECT COUNT(DISTINCT zdlgeom)
            FROM joint_geom_arret_zdl''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT COUNT(DISTINCT cod_tlb_arrt_sttn)
            FROM stif_validation_unitaire''')
cur.fetchall()

In [ ]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()

In [ ]:
start_time=time.time()
cur.execute('''SELECT a.numr_cart,COUNT(DISTINCT b.zdlgeom) AS count
            FROM stif_validation_unitaire a,joint_geom_arret_zdl b
            WHERE EXTRACT(isodow FROM dat_vald) IN (6,7)
            AND a.cod_tlb_arrt_sttn = b.cod_tlb_arrt_sttn
            GROUP BY 1
            ORDER BY count desc
            ''')
num_uniq_stat_weekend=cur.fetchall()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time=time.time()
cur.execute('''SELECT a.numr_cart,COUNT(DISTINCT b.zdlgeom) AS count
            FROM stif_validation_unitaire a,joint_geom_arret_zdl b
            WHERE EXTRACT(isodow FROM dat_vald) IN (1,2,3,4,5)
            AND a.cod_tlb_arrt_sttn = b.cod_tlb_arrt_sttn
            GROUP BY 1
            ORDER BY count desc
            ''')
num_uniq_stat_weekday=cur.fetchall()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
num_uniq_stat_weekend=pd.DataFrame(num_uniq_stat_weekend)

In [ ]:
num_uniq_stat_weekend.columns=['numr_cart','count']

In [ ]:
num_uniq_stat_weekend.to_csv('num_uniq_stat_weekend.csv')

In [ ]:
num_uniq_stat=pd.read_csv('num_uniq_station.csv')

In [ ]:
num_uniq_stat

In [ ]:
num_uniq_stat.to_csv('num_uniq_station.csv')

In [ ]:
num_uniq_stat.describe()

In [ ]:
plt.boxplot([num_uniq_stat['num_station_weekday'],num_uniq_stat['num_station_weekend']],widths=0.6)
plt.ylabel('count')
plt.xlabel('weekday                  /                    weekend')
plt.title('Number of unique stations')
plt.show()

In [ ]:
sns.distplot(num_uniq_stat['num_station_weekday'])
plt.ylabel('Frequency')
plt.title('Number of unique stations in weekday')
plt.savefig('num_uniq_stat_weekday.png',format='png',dpi=1200)

In [ ]:
sns.distplot(num_uniq_stat['num_station_weekend'])
plt.ylabel('Frequency')
plt.title('Number of unique stations in weekend')
plt.savefig('num_uniq_stat_weekend.png',format='png',dpi=1200)

In [ ]:
tips=sns.load_dataset("tips")

In [ ]:
tips

In [ ]:
sns.boxplot(y=('num_station_weekday'),data=num_uniq_stat)

# Total days of everyone

In [ ]:
cur.execute("""SELECT numr_cart,COUNT(DISTINCT dat_vald) AS count
            FROM stif_validation_unitaire
            GROUP BY 1
            ORDER BY count desc""")
num_days=cur.fetchall()

num_of_day = pd.DataFrame(num_days)
num_of_day.to_csv('num_of_day.csv')

In [ ]:
num_of_day

In [ ]:
plt.boxplot(num_of_day[1])
plt.ylabel('days')
plt.show()

# Most(top 2) frequent station 

In [ ]:
cur.execute("""
            SELECT numr_cart,count,Rowid,c.cod_tlb_arrt_sttn,d.libl_long_arrt_sttn
            FROM(
            SELECT numr_cart,cod_tlb_arrt_sttn,count,Rowid
            FROM(
            SELECT *,ROW_NUMBER() OVER (PARTITION BY numr_cart ORDER BY count desc) AS Rowid
            FROM(
            SELECT DISTINCT numr_cart,cod_tlb_arrt_sttn,COUNT(cod_tlb_arrt_sttn) AS count
            FROM stif_validation_unitaire
            GROUP BY 1,2
            ) AS a
            ) AS b
            WHERE Rowid < 3 
            ORDER BY numr_cart
            ) AS c, stif_arret_station_physique d
            WHERE d.cod_tlb_arrt_sttn = c.cod_tlb_arrt_sttn
            AND d.libl_long_arrt_sttn != 'Inconnu'
            ORDER BY numr_cart
            """)
count_station=cur.fetchall()

In [ ]:
count_station=pd.DataFrame(count_station)
count_station.columns=['numr_cart','count','Order','cod_tlb_arrt_sttn','libl_long_arrt_sttn']
count_station.to_csv('count_station.csv')

In [ ]:
count_station

In [ ]:
count_station[count_station['numr_cart'] == 1853439815032428009]

In this table there are some people that have a record like above, they have equal validation times for a lot of stations, for these records maybe should delete them

In [ ]:
cur.execute("""
            SELECT numr_cart,count,Rowid,c.cod_tlb_arrt_sttn,d.libl_long_arrt_sttn
            FROM(
            SELECT numr_cart,cod_tlb_arrt_sttn,count,Rowid
            FROM(
            SELECT *,ROW_NUMBER() OVER (PARTITION BY numr_cart ORDER BY count desc) AS Rowid
            FROM(
            SELECT DISTINCT numr_cart,cod_tlb_arrt_sttn,COUNT(cod_tlb_arrt_sttn) AS count
            FROM(
            SELECT * 
            FROM stif_validation_unitaire
            WHERE EXTRACT(isodow FROM dat_vald) IN (1,2,3,4,5)
            ) AS w
            GROUP BY 1,2
            ) AS a
            ) AS b
            WHERE Rowid < 3 
            ORDER BY numr_cart
            ) AS c, stif_arret_station_physique d
            WHERE d.cod_tlb_arrt_sttn = c.cod_tlb_arrt_sttn
            AND d.libl_long_arrt_sttn != 'Inconnu'
            ORDER BY numr_cart
            """)
count_station_weekday=cur.fetchall()

In [ ]:
cur.execute("""
            SELECT numr_cart,count,Rowid,c.cod_tlb_arrt_sttn,d.libl_long_arrt_sttn
            FROM(
            SELECT numr_cart,cod_tlb_arrt_sttn,count,Rowid
            FROM(
            SELECT *,ROW_NUMBER() OVER (PARTITION BY numr_cart ORDER BY count desc) AS Rowid
            FROM(
            SELECT DISTINCT numr_cart,cod_tlb_arrt_sttn,COUNT(cod_tlb_arrt_sttn) AS count
            FROM(
            SELECT * 
            FROM stif_validation_unitaire
            WHERE EXTRACT(isodow FROM dat_vald) IN (6,7)
            ) AS w
            GROUP BY 1,2
            ) AS a
            ) AS b
            WHERE Rowid < 3 
            ORDER BY numr_cart
            ) AS c, stif_arret_station_physique d
            WHERE d.cod_tlb_arrt_sttn = c.cod_tlb_arrt_sttn
            AND d.libl_long_arrt_sttn != 'Inconnu'
            ORDER BY numr_cart
            """)
count_station_weekend=cur.fetchall()

In [ ]:
count_station_weekday=pd.DataFrame(count_station_weekday)
count_station_weekend=pd.DataFrame(count_station_weekend)
count_station_weekday.columns=['numr_cart','count','order','cod_tlb_arrt_sttn','libl_long_arrt_sttn']
count_station_weekend.columns=['numr_cart','count','order','cod_tlb_arrt_sttn','libl_long_arrt_sttn']

In [ ]:
count_station_weekday.to_csv('count_station_weekday.csv')
count_station_weekend.to_csv('count_station_weekend.csv')

In [ ]:
count_station_sp=count_station_weekday.join(count_station_weekend,lsuffix='numr_cart',rsuffix='numr_cart',how='outer')
count_station_sp.columns=['numr_cart','count_weekday','order_weekday','cod_weekday','libl_weekday','numr_carte','count_weekend','order_weekend','cod_weekend','libl_weekend']
count_station_sp=count_station_sp.drop(columns=['numr_carte'])

In [ ]:
count_station_sp.to_csv('count_station_sp.csv')

In [ ]:
count_station_sp

In [ ]:
count_station_sp['libl_weekday'].value_counts()

In [ ]:
count_station_sp[count_station_sp.order_weekday==1]['libl_weekday'].value_counts()

In [ ]:
station1_weekday=count_station_sp[count_station_sp.order_weekday==1]['libl_weekday'].value_counts().to_frame().reset_index()
station1_weekday.columns=['station','count']
station1_weekend=count_station_sp[count_station_sp.order_weekend==1]['libl_weekend'].value_counts().to_frame().reset_index()
station1_weekend.columns=['station','count']
station2_weekday=count_station_sp[count_station_sp.order_weekday==2]['libl_weekday'].value_counts().to_frame().reset_index()
station2_weekday.columns=['station','count']
station2_weekend=count_station_sp[count_station_sp.order_weekend==2]['libl_weekend'].value_counts().to_frame().reset_index()
station2_weekend.columns=['station','count']

In [ ]:
sns.barplot(y='station',x='count',data=station1_weekday[0:10])
plt.title('Top 10 most frequent stations during weekday')

In [ ]:
sns.barplot(y='station',x='count',data=station1_weekend[0:10])
plt.title('Top 10 most frequent stations during weekend')

In [ ]:
sns.barplot(y='station',x='count',data=station2_weekday[0:10])
plt.title('Top 10 second most frequent stations during weekday')

In [ ]:
sns.barplot(y='station',x='count',data=station2_weekend[0:10])
plt.title('Top 10 second most frequent stations during weekend')

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald
            FROM stif_validation_unitaire
            ORDER BY heur_vald LIMIT 1''')
cur.fetchall()

# First(and last) three validations for each person each day

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            ) AS w
            WHERE w.r < 4
            ''')
first3_validation=cur.fetchall()

In [ ]:
first3_validation=pd.DataFrame(first3_validation)
first3_validation.to_csv('first3_validation.csv')

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald desc) AS r
            FROM stif_validation_unitaire
            ) AS w
            WHERE w.r < 4
            ''')
last3_validation=cur.fetchall()
last3_validation=pd.DataFrame(last3_validation)
last3_validation.to_csv('last3_validation.csv')

In [ ]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()

# Time difference

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,cod_tlb_arrt_sttn
            ,heur_vald - lag(heur_vald) OVER (ORDER BY numr_cart,dat_vald,heur_vald) AS interval
            FROM stif_validation_unitaire
            WHERE numr_cart = '4362165769991'
            ''')
td=cur.fetchall()

In [ ]:
td=pd.DataFrame(td)

In [ ]:
td

In [ ]:
cur.execute('''SELECT *
            FROM stif_validation_unitaire
            WHERE numr_cart = '4362165769991'
            ''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,cod_tlb_arrt_sttn
            ,heur_vald - lag(heur_vald) OVER (ORDER BY numr_cart,dat_vald,heur_vald) AS interval
            FROM stif_validation_unitaire
            ''')
td=cur.fetchall()

In [ ]:
td=pd.DataFrame(td)

In [ ]:
td.columns=['']

# Entropy 

In [ ]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()

In [ ]:
start_time=time.time()
cur.execute('''CREATE TABLE count_each AS
            SELECT numr_cart,zdlgeom,AVG(count) AS avg
            FROM(
            SELECT numr_cart,dat_vald,b.zdlgeom,COUNT(b.zdlgeom) AS count
            FROM stif_validation_unitaire a,joint_geom_arret_zdl b
            WHERE b.cod_tlb_arrt_sttn = a.cod_tlb_arrt_sttn
            GROUP BY 1,2,3
            ) AS w
            GROUP BY 1,2
            ''')
count_each=cur.fetchall()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
count_each=pd.DataFrame(count_each)
count_each.columns=['numr_cart','cod','count']

In [ ]:
count_each=pd.merge(count_each,count_each.groupby(['numr_cart'])['count'].sum().reset_index(),on='numr_cart')

In [ ]:
count_each['p']=count_each['count_x']/count_each['count_y']

In [ ]:
count_each['p']=count_each['p'].astype(float)

In [ ]:
count_each

In [ ]:
entropy=count_each.groupby(['numr_cart','p'])['p'].apply(lambda x:
                                          -x * np.log2(x)).reset_index()

In [ ]:
count_each['entropy']=entropy['p']

In [ ]:
ent=count_each.groupby('numr_cart')['entropy'].sum().reset_index()

In [ ]:
ent

In [ ]:
count_each

In [ ]:
sns.distplot(ent['entropy'])
plt.xlabel('Entropy')
plt.ylabel('Frequency')
plt.title('Distribution of entropy')

# Spatial-temporal entropy

In [3]:
cur.execute('''SELECT numr_cart,spatiotemporel_entropie
            FROM user_spatiotemporel_stats''')
Ent=cur.fetchall()
Ent=pd.DataFrame(Ent)

In [4]:
Ent[2]=Ent[1]

In [5]:
Ent

,0,1,2
0,4362165769991,8.065744,8.065744
1,60155522867454,6.686903,6.686903
2,73681551316104,7.209689,7.209689
3,99308585375234,4.858564,4.858564
4,166492329987075,3.277613,3.277613
5,289415928808559,7.447779,7.447779
6,294479101194672,4.363713,4.363713
7,304181386091598,5.979992,5.979992
8,311065912768062,4.772084,4.772084
9,344822678553909,3.916127,3.916127


In [6]:
Ent[1].describe()

count    100000.000000
mean          5.530564
std           1.484527
min          -0.000000
25%           4.681728
50%           5.713277
75%           6.581241
max          10.097748
Name: 1, dtype: float64

In [ ]:
sns.distplot(Ent[1])
plt.xlabel('Entropy')
plt.ylabel('Percentage')
plt.title('Distribution of entropy')
#plt.axvline(x=4.681728,color='g',linestyle='--')
#plt.axvline(x=6.581241,color='g',linestyle='--')
#plt.savefig('entropy.png', format='png', dpi=1200)

In [7]:
from sklearn.cluster import KMeans
kmeans_ent = KMeans(n_clusters=3, random_state=0).fit(Ent[[1,2]])

In [8]:
Ent[3]=kmeans_ent.predict(Ent[[1,2]])

In [ ]:
ent2=max(Ent[Ent[3]==2][1])
ent1=min(Ent[Ent[3]==2][1])
ax=sns.distplot(Ent[1],color='green')
kx,ky=ax.lines[0].get_data()
plt.axvline(x=ent1,color='blue',linestyle='--')
plt.axvline(x=ent2,color='blue',linestyle='--')
ax.fill_between(kx,ky,where=(kx<ent1),color='green')
ax.fill_between(kx,ky,where=(kx>ent2),color='red')
plt.xlabel('Entropy')
plt.ylabel('Frequency')
plt.title('Grouped entropy')
plt.annotate('13.135%',xy=(0.8,0.28))
plt.annotate('43.508%',xy=(4.2,0.28))
plt.annotate('43.357%',xy=(7.8,0.28))
plt.savefig('ent_gp.png',format='png',dpi=1200)
plt.show()

In [ ]:
for i in range(3):
    print(len(Ent[Ent[3]==i])/(len(Ent)))

# List of people valide

In [9]:
cur.execute('''SELECT a.numr_cart,a.spatiotemporel_entropie,COUNT(b.numr_cart) AS count,COUNT(DISTINCT b.cod_tlb_arrt_sttn) AS counts
            FROM user_spatiotemporel_stats a, stif_validation_unitaire b
            WHERE a.numr_cart=b.numr_cart
            GROUP BY 1,2
            ORDER BY 2''')
list_num=cur.fetchall()
list_num=pd.DataFrame(list_num)

In [10]:
list_num=list_num[list_num[1]!=-0.000000].reset_index(drop=True)

In [11]:
list_num=pd.merge(list_num,Ent,on=0)

In [12]:
list_num

,0,1_x,2_x,3_x,1_y,2_y,3_y
0,1121724605935971953,0.200622,32,2,0.200622,0.200622,0
1,1836155104747366812,0.413817,12,1,0.413817,0.413817,0
2,15531654811174045,0.543564,8,2,0.543564,0.543564,0
3,1562146471669370192,0.644327,62,1,0.644327,0.644327,0
4,1387099253037193529,0.650022,6,1,0.650022,0.650022,0
5,1639312495467408742,0.721928,5,2,0.721928,0.721928,0
6,1712265640942072664,0.721928,5,1,0.721928,0.721928,0
7,114652688218038015,0.787341,56,2,0.787341,0.787341,0
8,503998609768509820,0.811278,4,1,0.811278,0.811278,0
9,1369502566583485156,0.811278,4,2,0.811278,0.811278,0


In [ ]:
i=list_num[(list_num['2_x']>20)&(list_num['1_x']>3.5)&(list_num['3_y']==0)][0].tolist()

In [ ]:
list_num[(list_num[0].isin(i))&(list_num['2_x']>200)&(list_num['3_x']>5)]

# Time interval

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = %s
            ) AS w
            ''',
            ['673006919441222358'])
record=cur.fetchall()
record=pd.DataFrame(record)

In [ ]:
record[5]=record[1].apply(lambda x: x.isoweekday())
record=record[[0,1,5,2,3,4]]

In [ ]:
record[6]=record[2].apply(lambda x: 3 if x.hour<6 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))

In [ ]:
record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)

In [ ]:
plt.scatter(x=record_weekday[2].tolist(),y=record_weekday[2].tolist(),s=5,c='blue')
plt.title('Time records')
#plt.savefig('hours.png',format='png',dpi=1200)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
X=record_weekday[2].apply(lambda x: 
                x.hour * 3600 + x.minute * 60 + x.second).values.reshape(-1,1)
silhouette=[]
K=range(2,13)
for i in K:
    kmeans = KMeans(n_clusters=i, random_state=0).fit(X)
    Z=kmeans.predict(X)
    silhouette.append(silhouette_score(X,Z))
    
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('The Silhouette score showing the optimal k')
#plt.savefig('silhouette.png',format='png',dpi=1200)
plt.show()

In [ ]:
np.argmax(silhouette)+2

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
X=record_weekday[2].apply(lambda x: 
                    x.hour * 3600 + x.minute * 60 + x.second).values.reshape(-1,1)
distortions=[]
K=range(2,13)
for k in K:
    kmeans=KMeans(n_clusters=k).fit(X)
    kmeans.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeans.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
    
plt.plot(K, distortions, 'bx-')
plt.plot(3,distortions[1],markersize=500,color='red')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
#plt.savefig('Elbow.png',format='png',dpi=1200)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
kmeans_weekday = KMeans(n_clusters=9, random_state=0).fit(record_weekday[2].apply(lambda x: 
                x.hour * 3600 + x.minute * 60 + x.second).values.reshape(-1,1))

In [ ]:
Z_weekday=kmeans_weekday.predict(record_weekday[2].apply(lambda x: 
                x.hour * 3600 + x.minute * 60 + x.second).values.reshape(-1,1))
Z_weekday

In [ ]:
record_weekday[6]=Z_weekday

In [ ]:
record_weekday=record_weekday[[0,1,5,2,6,3,4]]
#record_weekday.columns=['num_cart','dat','weekday','heur','time_interval','cod_arrt','order']

In [ ]:
plt.scatter(x=record_weekday[2].tolist(),y=record_weekday[2].tolist(),s=5,c=record_weekday[6])
plt.title('Time records')
#plt.savefig('hours_dis.png',format='png',dpi=1200)

In [ ]:
#select a best discretization by minimizing the entropy

In [ ]:
prop=record.groupby([5,3]).count().reset_index()

In [ ]:
pp=[0]*(len(prop))
for i in range(len(prop)):
    pp[i]=((prop[4][i])/(record.groupby(5)[0].count()[prop[5][i]]))

In [ ]:
prop[6]=pp

In [ ]:
prop[6].apply(lambda x: -x * np.log2(x)).sum()/len(prop)

# Distribution in time interval

In [ ]:
sns.distplot(record[record[6]==0][2].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second))

In [ ]:
from scipy import stats

k=len(record[6].unique())
kernel=[0] * k
for i in range(k):
    hs=record[record[6]==i][2].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    kernel[i]=stats.gaussian_kde(hs)

In [ ]:
kernel

In [ ]:
kernel[0].integrate_box_1d(20000,25000)

# Geographic

In [ ]:
tuple(record[3].unique())

In [ ]:
cur.execute('''SELECT cod_tlb_arrt_sttn,libl_long_arrt_sttn,coor_x_phsq,coor_y_phsq,
            FROM stif_arret_station_physique 
            WHERE cod_tlb_arrt_sttn IN %s
            ''',
            tuple(record[3].unique()))
location=cur.fetchall()
location=pd.DataFrame(location)

In [ ]:
location

In [ ]:
from pyproj import Proj, transform
inn=Proj(init='epsg:27561')
out=Proj(init='epsg:4326')
for i in range(len(location)):
    location[3][i],location[2][i]=transform(inn,out,location[2][i],location[3][i])

In [ ]:
locationlist=location[[2,3]].values.tolist()

In [ ]:
# function to compute distances between trips
def distance_on_geoid(df):
    # convert degrees to radians
    df['lat'] = df['lat'] * math.pi / 180
    df['lon'] = df['lon'] * math.pi / 180

    df['lat_shift'] = df['lat_shift'] * math.pi / 180
    df['lon_shift'] = df['lon_shift'] * math.pi / 180
    
    # radius of earth in meters
    r = 6378100
    rho1 = r * math.cos(df['lat'])
    z1 = r * math.sin(df['lat'])
    x1 = rho1 * math.cos(df['lon'])
    y1 = rho1 * math.sin(df['lon'])
    
    rho2 = r * math.cos(df['lat_shift'])
    z2 = r * math.sin(df['lat_shift'])
    x2 = rho2 * math.cos(df['lon_shift'])
    y2 = rho2 * math.sin(df['lon_shift'])
    
    # dot product
    dot = (x1*x2 + y1*y2 + z1*z2)
    cos_theta = dot /(r*r)
    if cos_theta > 1:
        cos_theta = 1
    elif cos_theta < -1 :
        cos_theta = -1
    #return cos_theta
    theta = math.acos(np.fabs(cos_theta))    
    return r*theta

In [ ]:
import folium
from folium.plugins import MarkerCluster
mp=folium.Map(location=[48.8738818,2.3853162])
marker_cluster=MarkerCluster().add_to(mp)
for i in range(len(location)):
    folium.Marker(locationlist[i]).add_to(marker_cluster)
mp

In [ ]:
location[3]=0
for i in range(len(location)):
    location[3][i]=record[record[3]==location[0][i]][3].count()

In [ ]:
from scipy.stats import gaussian_kde
plt.scatter(x=location[2],y=location[1])
plt.xlabel('coor_x')
plt.ylabel('coor_y')
#plt.savefig('location.png',format='png',dpi=1200)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
X=location[[1,2]]
score=[]
K=range(2,len(location))
for i in K:
    kmeans = KMeans(n_clusters=i, random_state=0).fit(X)
    Z=kmeans.predict(X)
    score.append(silhouette_score(X,Z))
    
plt.plot(K, score, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('The Silhouette score for each k')
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=35).fit(X)
location[4]=kmeans.predict(X)

In [ ]:
plt.scatter(x=location[1],y=location[2],c=location[4])

# Markov Model Accuracy

In [ ]:
##################################################################################
# Intergrated Function for markov model
##################################################################################

In [ ]:
#Markov model with fixed time interval
def transition_matrix(transitions,n):
    n = n #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M
def markov_model(record,Y):    
    k=len(record[6].unique())   #number of time intervals
    M=[0] * k                   #transition matrice
    for i in range(k):
        M[i]=transition_matrix(record[record[6]==i][3].map(Y).values,len(Y)) #Transition matrix   
    return M
def markov_model_accuracy(numr_cart,t,f):
    cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = %s
            ) AS w
            ''',
            [numr_cart])
    record=cur.fetchall()
    record=pd.DataFrame(record)
    record[5]=record[1].apply(lambda x: x.isoweekday())
    record=record[[0,1,5,2,3,4]]
    record[6]=record[2].apply(lambda x: 3 if x.hour<5 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))
    record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
    record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)
    train_len=(2*len(record_weekday))//3
    train=record_weekday[0:train_len]
    test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)
    Y=pd.Series(range(len(record_weekday[3].unique())),index=record_weekday[3].unique()) #mapping 
    M=markov_model(train,Y)
    #test 
    #t=[0]*4
    #f=[0]*4
    for j in range(0,4):
        data=test[test[6]==j].reset_index(drop=True)
        for i in data.index[:-1]:
            state=data[3].map(Y).fillna(value=0).astype(int)[i]
            time=data[6][i]
            pred=np.argmax(M[time][state])
            nextstate=data[3].map(Y).fillna(value=0).astype(int)[i+1]
            if pred==nextstate:
                t[j]=t[j]+1
            else:
                f[j]=f[j]+1
    return t,f

In [ ]:
list_num

In [ ]:
i=list_num[(list_num['2_x']>200)&(list_num['1_x']>2)&(list_num['3_x']==0)][0].tolist()

In [ ]:
list_num[(list_num[0].isin(i))&(list_num['2_x']>200)&(list_num['3_y']>3)]

In [ ]:
inn=list_num[(list_num[0].isin(i))&(list_num['2_x']>200)&(list_num['3_x']>5)][0].tolist()

In [ ]:
start_time=time.time()
t=[0]*4
f=[0]*4
for k in range(50):
    markov_model_accuracy(str(inn[k]),t,f)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
t

In [ ]:
f

In [ ]:
[t[i]/(t[i]+f[i]) for i in range(len(t))] #accuracy

# Markov Model

In [ ]:
def transition_matrix(transitions,n):
    n = n #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M
def markov_model(record):    
    k=len(record[6].unique())   #number of time intervals
    M=[0] * k                   #transition matrice
    Y=pd.Series(range(len(record[3].unique())),index=record[3].unique()) #mapping 
    for i in range(k):
        M[i]=transition_matrix(record[record[6]==i][3].map(Y).values,len(Y)) #Transition matrix   
    return M,Y

In [ ]:
train_len=(2*len(record_weekday))//3
train=record_weekday[0:train_len]
test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)

In [ ]:
M,Y=markov_model(train)

In [ ]:
for i in range(1,len(M)+1):
    plt.subplot(2,2,i)
    plt.imshow(M[i-1])
    plt.title(i)
    plt.axis('tight')
    #plt.tight_layout()

plt.suptitle('Markov Model Transition matrix for each time interval(weekday)')
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.85,hspace=0.4,wspace=0.25)
cax = plt.axes([0.85, 0.1, 0.025, 0.8])
plt.colorbar(cax=cax)
plt.savefig('markov_model.png', format='png', dpi=1200)
plt.show()

In [ ]:
#test
ti=4
t=[0]*ti
f=[0]*ti
for j in range(0,ti):
    data=test[test[6]==j].reset_index()
    for i in data.index[:-1]:
        state=data[3].map(Y).fillna(value=0).astype(int)[i]
        time=data[6][i]
        pred=M[time][state].index(max(M[time][state]))
        nextstate=data[3].map(Y).fillna(value=0).astype(int)[i+1]
        if pred==nextstate:
            t[j]=t[j]+1
        else:
            f[j]=f[j]+1

In [ ]:
t

In [ ]:
f

In [ ]:
[t[i]/(t[i]+f[i]) for i in range(len(t))] #accuracy

In [ ]:
sum(t)/(sum(t)+sum(f)-4)

In [ ]:
record[5].value_counts()

In [ ]:
i=2
y=pd.Series(range(len(record[3].unique())),index=record[3].unique())

In [ ]:
y

In [ ]:
record[record[5]==2][3].map(y).values

In [ ]:
def transition_matrix(transitions):
    n = 1+max(transitions) #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [ ]:
def transition_matrix(transitions):
    n = 1+max(transitions) #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def markov_model(record):    
    M=[0] * 4
    Y=[0] * 4
    
    for i in range(0,4):
        Y[i]=pd.Series(range(len(record[record[5]==i][3].unique())),index=record[record[5]==i][3].unique())
        M[i]=transition_matrix(record[record[5]==i][3].map(Y[i]).values)
        
    return M,Y

In [ ]:
P=transition_matrix(record[record[5]==2][3].map(y).values)

In [ ]:
P[1]

# 3 states markov model accuracy

In [ ]:
#Markov model with fixed time interval
def markov_model3_accuracy(numr_cart,t,f):
    cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = %s
            ) AS w
            ''',
            [numr_cart])
    record=cur.fetchall()
    record=pd.DataFrame(record)
    record[5]=record[1].apply(lambda x: x.isoweekday())
    record=record[[0,1,5,2,3,4]]
    record[6]=record[2].apply(lambda x: 3 if x.hour<5 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))
    fst=record[3].value_counts().index.tolist()[0:2]
    record[3]=record[3].apply(lambda x: 1 if x== fst[0] else (2 if x== fst[1] else 3))
    record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
    record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)
    train_len=(2*len(record_weekday))//3
    train=record_weekday[0:train_len]
    test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)
    M,Y=markov_model(train)
    #test 
    #t=[0]*4
    #f=[0]*4
    for j in range(0,4):
        data=test[test[6]==j].reset_index(drop=True)
        for i in data.index[:-1]:
            state=data[3].map(Y).fillna(value=0).astype(int)[i]
            time=data[6][i]
            pred=np.argmax(M[time][state])
            nextstate=data[3].map(Y).fillna(value=0).astype(int)[i+1]
            if pred==nextstate:
                t[j]=t[j]+1
            else:
                f[j]=f[j]+1
    return t,f

In [ ]:
i=list_num[(list_num[2]>20)&(list_num[1]>6.6)&(list_num[1]<10)][0].tolist()

In [ ]:
start_time=time.time()
t=[0]*4
f=[0]*4
for k in range(100):
    markov_model3_accuracy(str(i[k]),t,f)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
t

In [ ]:
f

In [ ]:
[t[i]/(t[i]+f[i]) for i in range(len(t))] #accuracy

# Markov Model for 3 states

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = '1344228075588831'
            ) AS w
            ''')
record=cur.fetchall()
record=pd.DataFrame(record)

In [ ]:
record[3].value_counts().index.tolist()

In [ ]:
plt.bar(range(len(record[3].value_counts().values)),height=record[3].value_counts().values)
plt.title('Bar plot for stations\' frequency')
plt.ylabel('Frequency')
plt.xlabel('Stations')
plt.savefig('bar_freq.png', format='png', dpi=1200)

In [ ]:
record[3]=record[3].apply(lambda x: 1 if x=='0030050170601' else (2 if x=='0030050170503' else 3))

In [ ]:
record

In [ ]:
def transition_matrix(transitions,n):
    n = n #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M
def markov_model(record):    
    M=[0] * 4
    Y=pd.Series(range(len(record[3].unique())),index=record[3].unique()) #mapping 
    
    for i in range(0,4):
        M[i]=transition_matrix(record[record[5]==i][3].map(Y).values,len(Y)) #Transition matrix
        
    return M,Y

In [ ]:
record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)

In [ ]:
train_len=(2*len(record_weekday))//3
train=record_weekday[0:train_len]
test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)

In [ ]:
M,Y=markov_model(train)

In [ ]:
for i in range(1,5):
    plt.subplot(2,2,i)
    plt.imshow(M[i-1])

plt.suptitle('3 States Markov Model Transition matrix for each time interval')
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
cax = plt.axes([0.85, 0.1, 0.025, 0.8])
plt.colorbar(cax=cax)
plt.savefig('markov_model_3.png',format='png',dpi=1200)
plt.show()

In [ ]:
#test
t=[0]*4
f=[0]*4
for j in range(0,4):
    data=test[test[5]==j].reset_index()
    for i in data.index[:-1]:
        state=data[3].map(Y).astype(int)[i]
        time=data[5][i]
        pred=M[time][state].index(max(M[time][state]))
        nextstate=data[3].map(Y).astype(int)[i+1]
        if pred==nextstate:
            t[j]=t[j]+1
        else:
            f[j]=f[j]+1

In [ ]:
t

In [ ]:
f

In [ ]:
[t[i]/(t[i]+f[i]) for i in range(len(t))] #accuracy

# First Validations prediction

In [ ]:
def first_val(record):
    firstvt=record[record[4]==1].reset_index()
    heur=firstvt[2].apply(lambda x: x.hour)
    l=len(heur)
    test=l//3
    train=l-test
    #train set
    X=[0] * train
    Y=[0] * train
    for i in range(5,train+5):
        Y[i-5]=heur[i]
        X[i-5]=heur[(i-5):i]
    #SVR
    from sklearn.svm import SVR
    svr_rbf=SVR(kernel='rbf', C=1e3, degree=2)
    model=svr_rbf.fit(X,Y)
    #Predict
    y_rbf=[0] * l
    y_rbf[0:(train+5)]=heur[0:(train+5)]
    x=[0] * l
    x[0:train]=X[0:train]
    for i in range(train,l-5):
        y_rbf[i+5]=model.predict(x[(i-1):i])[0]
        x[i]=y_rbf[i:(i+5)]
    
    plt.plot(y_rbf,label='prediction')
    plt.plot(heur,label='real')
    plt.xlabel('day')
    plt.ylabel('hour')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = '1344228075588831'
            ) AS w
            ''')
record=cur.fetchall()

In [ ]:
record=pd.DataFrame(record)
record

In [ ]:
first_val(record)

In [ ]:
firstvt=record[record[4]==1].reset_index()

In [ ]:
firstvt

In [ ]:
firstvt[firstvt[1].apply(lambda x: x.weekday())==5]

In [ ]:
firstvt[2].apply(lambda x: x.hour)

In [ ]:
plt.plot(firstvt[1],firstvt[2].apply(lambda x: x.hour))
plt.show()

In [ ]:
heur=firstvt[2].apply(lambda x: x.hour)

In [ ]:
heur

In [ ]:
#train set
X=[0] * 45
Y=[0] * 45
for i in range(5,50):
    Y[i-5]=heur[i]
    X[i-5]=heur[(i-5):i]

In [ ]:
#SVR
from sklearn.svm import SVR
svr_rbf=SVR(kernel='rbf', C=1e3, degree=2)
model=svr_rbf.fit(X,Y)

In [ ]:
model.predict(X[44:45])[0]

In [ ]:
#Predict
y_rbf=[0] * 63
y_rbf[0:50]=heur[0:50]
x=[0] * 63
x[0:45]=X[0:45]
for i in range(45,58):
    y_rbf[i+5]=model.predict(x[(i-1):i])[0]
    x[i]=y_rbf[i:(i+5)]
    

In [ ]:
y_rbf

In [ ]:
    plt.plot(y_rbf,label='prediction')
    plt.plot(heur,label='real')
    plt.xlabel('day')
    plt.ylabel('hour')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
record[2].tolist()

# Hidden Markov Model 

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = '1344228075588831'
            ) AS w
            ''')
record=cur.fetchall()
record=pd.DataFrame(record)

In [ ]:
record[6]=record[2].apply(lambda x: 3 if x.hour<6 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))

In [ ]:
record[7]=0
for i in range(1,len(record)-1):
    if record[1][i]==record[1][i+1]:
        record[7][i]=2*3600
    elif record[1][i]==record[1][i-1]:
        record[7][i]=(record[2][i].hour*3600+record[2][i].minute*60+record[2][i].second)-(record[2][i-1].hour*3600+record[2][i-1].minute*60+record[2][i-1].second)
    else:
        record[7][i]=(record[2][i].hour*3600+record[2][i].minute*60+record[2][i].second)-(record[2][i-1].hour*3600+record[2][i-1].minute*60+record[2][i-1].second)+24*3600

record[7][len(record)-1]=(record[2][len(record)-1].hour*3600+record[2][len(record)-1].minute*60+record[2][len(record)-1].second)-(record[2][len(record)-2].hour*3600+record[2][len(record)-2].minute*60+record[2][len(record)-2].second)

In [ ]:
record[7].describe()

In [ ]:
record

In [ ]:
record[3].value_counts().index.tolist()

In [ ]:
record[3]=record[3].apply(lambda x: 1 if x=='0030050170601' else (2 if x=='0030050170503' else 3))

In [ ]:
record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)

In [ ]:
Y=pd.Series(range(len(record_weekday[3].unique())),index=record_weekday[3].unique())
record_weekday[3]=record_weekday[3].map(Y).values

In [ ]:
train_len=(2*len(record_weekday))//3
train=record_weekday[0:train_len]
test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)

In [ ]:
test

In [ ]:
from hmmlearn import hmm
hmm=hmm.GaussianHMM(n_components=len(record_weekday[3].unique()), covariance_type="diag", n_iter=1000)
hmm.fit(train[[7,6]].values)

In [ ]:
hmm.transmat_

In [ ]:
plt.imshow(hmm.transmat_)
plt.colorbar()
plt.show()

In [ ]:
hmm.predict(test[[7,6]])+1

In [ ]:
test[3].values

In [ ]:
t=0
f=0
for i in range(len(test)):
    pred=(hmm.predict(test[[7,5]])+1)[i]
    if pred==test[3][i]:
        t=t+1
    else:
        f=f+1

In [ ]:
t/(t+f)

In [ ]:
for i in range(0,4):
    plt.subplot(2,2,i+1)
    y=pd.Series(range(len(record[record[5]==i][3].unique())),index=record[record[5]==i][3].unique())
    tr=record[record[5]==i][3].map(y).values
    ob=[0] * (len(tr)-1)
    for k in range(len(tr)-1):
        ob[k]=tr[k:(k+2)]
    
    from hmmlearn import hmm
    hmm=hmm.GaussianHMM(n_components=len(record[record[5]==i+1][3].unique())+1, covariance_type="full", n_iter=100)
    hmm.fit(ob)
    plt.imshow(hmm.transmat_)
    
plt.suptitle('HMM Transition matrix for each time interval')
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
cax = plt.axes([0.85, 0.1, 0.025, 0.8])
plt.colorbar(cax=cax)
plt.show()

In [ ]:
>>> import numpy as np
>>> from hmmlearn import hmm
>>> np.random.seed(42)

>>> model = hmm.GaussianHMM(n_components=3, covariance_type="full")
>>> model.startprob_ = np.array([0.6, 0.3, 0.1])
>>> model.transmat_ = np.array([[0.7, 0.2, 0.1],
...                             [0.3, 0.5, 0.2],
...                             [0.3, 0.3, 0.4]])
>>> model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
>>> model.covars_ = np.tile(np.identity(2), (3, 1, 1))
>>> X, Z = model.sample(100)

# HMM accuracy

In [ ]:
#Markov model with fixed time interval
def hmm_accuracy(numr_cart,t,f):
    cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = %s
            ) AS w
            ''',
            [numr_cart])
    record=cur.fetchall()
    record=pd.DataFrame(record)
    record[5]=record[1].apply(lambda x: x.isoweekday())
    record=record[[0,1,5,2,3,4]]
    record[6]=record[2].apply(lambda x: 3 if x.hour<5 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))
    fst=record[3].value_counts().index.tolist()[0:2]
    record[3]=record[3].apply(lambda x: 1 if x== fst[0] else (2 if x== fst[1] else 3))
    record[7]=0
    for i in range(1,len(record)):
        if record[1][i]==record[1][i-1]:
            record[7][i]=(record[2][i].hour*3600+record[2][i].minute*60+record[2][i].second)-(record[2][i-1].hour*3600+record[2][i-1].minute*60+record[2][i-1].second)
        else:
            record[7][i]=2*60*60
    record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
    record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)
    train_len=(2*len(record_weekday))//3
    train=record_weekday[0:train_len]
    test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)
    from hmmlearn import hmm
    hmm=hmm.GaussianHMM(n_components=len(train[3].unique()), covariance_type="diag", n_iter=1000)
    hmm.fit(train[[7,5]].values)
    #test 
    #t=[0]*4
    #f=[0]*4
    for i in range(len(test)):
        pred=(hmm.predict(test[[7,5]])+1)[i]
        if pred==test[3][i]:
            t=t+1
        else:
            f=f+1
    return t,f

In [ ]:
i=list_num[(list_num['2_x']>200)&(list_num['1_x']>1)&(list_num[3]==0)][0].tolist()

In [ ]:
hmm_accuracy(str(i[1]),12,19)

In [ ]:
start_time=time.time()
t=0
f=0
for k in range(5):
    (m,n)=hmm_accuracy(str(i[k]),0,0)
    t=t+m
    f=f+n
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
t

In [ ]:
f

In [ ]:
t/(t+f)

# Dynamic Bayesian Network

# Random Forest

In [ ]:
cur.execute('''SELECT a.cod_tlb_arrt_sttn,
                      b.day_of_week,
                      CAST(b.public_holidays AS integer),
                      CAST(b.school_holidays AS integer),
                      CAST(b.bridging_days AS integer),
                      (EXTRACT(HOUR FROM a.heur_vald)*4+trunc(EXTRACT(MINUTE FROM a.heur_vald)/15)+1) AS timestep,
                      COUNT(*) AS count
            FROM stif_validation_unitaire a, daily_dimension b
            WHERE a.dat_vald=b.day
            GROUP BY 1,2,3,4,5,6
            ''')
flow=cur.fetchall()
count=pd.DataFrame(flow)

In [ ]:
count[5]=count[5].astype(int)

In [ ]:
count=count[~count[0].isin(['ND           ','*987         ','*97          ','*771         '])]

In [ ]:
count=count[~count[0].str.startswith('*')]

In [ ]:
mp=pd.Series(range(len(count[0].unique())),index=count[0].unique())

In [ ]:
count[0]=count[0].map(mp).values

In [ ]:
count

In [ ]:
cur.execute('''SELECT * 
            FROM stif_arret_station_physique
            WHERE cod_tlb_arrt_sttn='999'
            ''')
cur.fetchall()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor(n_estimators=64,max_depth=10)

In [ ]:
reg.fit(X=count[list(range(6))],y=count[6])

In [ ]:
reg.predict([[0,2,0,0,0,23]])

# Entire records

In [3]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()

In [ ]:
start_time=time.time()
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn
            FROM stif_validation_unitaire
            GROUP BY 1,2,3,4
            ''')
records=cur.fetchall()
records=pd.DataFrame(records)
print("--- %s seconds ---" % (time.time() - start_time))

In [14]:
records[5]=records[1].apply(lambda x: x.isoweekday()) #seperate weekday/weekend
records_weekend=records[records[5].isin([6,7])].reset_index(drop=True)
records_weekday=records[-records[5].isin([6,7])].reset_index(drop=True)

In [ ]:
records

# Global model

In [ ]:
def transition_matrix(transitions,n):
    n = n #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def markov_model(record,Y):    
    k=len(record[6].unique())   #number of time intervals
    M=[0] * k                   #transition matrice
    for i in range(k):
        M[i]=transition_matrix(record[record[6]==i][3].map(Y).values,len(Y)) #Transition matrix   
    return M

def distribution(record):
    from scipy import stats
    kernel=[0] * 4              #Gaussian KDE kernel
    for i in range(4):
        if i in (record[6].tolist()):
            kernel[i]=stats.gaussian_kde(record[record[6]==i][2].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second))
        else:
            kernel[i]=0        
    return kernel

def local_model(numr_cart):
    cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = %s
            ) AS w
            ''',
            [numr_cart])
    record=cur.fetchall()
    record=pd.DataFrame(record)  #get record
    
    record[5]=record[1].apply(lambda x: x.isoweekday())
    record=record[[0,1,5,2,3,4]] #seperate weekday/weekend
    
    record[6]=record[2].apply(lambda x: 3 if x.hour<5 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))
    #time interval
    
    record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
    record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)

    train_len=(2*len(record_weekday))//3
    train=record_weekday[0:train_len]
    test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)   #tarin/test set
    
    Y=pd.Series(range(len(record_weekday[3].unique())),index=record_weekday[3].unique()) #mapping 
    
    M=markov_model(train,Y)
    kernel=distribution(record_weekday)
    
    return Y,M,kernel

In [ ]:
num=list_num[(list_num['2_x']>20)&(list_num['1_x']>3.5)&(list_num['3_y']==0)][0].tolist()

In [ ]:
list_num[(list_num[0].isin(num))&(list_num['2_x']>200)&(list_num['3_x']>5)]

In [ ]:
numk=list_num[(list_num[0].isin(num))&(list_num['2_x']>200)&(list_num['3_x']>5)][0].tolist()

In [ ]:
local_model(str(numk[14]))

In [ ]:
numk[14]

In [ ]:
cur.execute('''SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,w.r
            FROM(
            SELECT numr_cart,dat_vald,heur_vald,cod_tlb_arrt_sttn,ROW_NUMBER() OVER (PARTITION BY numr_cart,dat_vald ORDER BY heur_vald) AS r
            FROM stif_validation_unitaire
            WHERE numr_cart = %s
            ) AS w
            ''',
            ['218653064136554507'])
record=cur.fetchall()
record=pd.DataFrame(record)
record

In [ ]:
record[5]=record[1].apply(lambda x: x.isoweekday())
record=record[[0,1,5,2,3,4]]

In [ ]:
record[6]=record[2].apply(lambda x: 3 if x.hour<6 else (0 if x.hour<11 else (1 if x.hour<16 else (2 if x.hour<21 else 3))))

In [ ]:
record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)

In [ ]:
record_weekday

In [ ]:
#
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()
start_time=time.time()

Y=[0]*len(numk)
M=[0]*len(numk)
kernel=[0]*len(numk)
for i in range(len(numk)):
    try:
        Y[i],M[i],kernel[i]=local_model(str(numk[i]))
    except:
        continue

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
len(Y)

In [ ]:
xx=[]
for i in range(len(Y)):
    if (type(kernel[i])==int)==True:
        xx.append(i)

In [ ]:
xx

In [ ]:
for i in xx:
    del Y[i]
    del M[i]
    del kernel[i]

In [ ]:
if ((arrta in Y[2])==True) and ((arrtb in Y[2])==True) and ((kernel[2][0]==0)==False):
    print('yes')
else:
    print('no')

In [ ]:
stt=8*60*60
end=8*60*60+30*60
ti=0   #time intervel
arrta='0030030210900'
arrtb='0030030110200'
flow=0
for i in range(len(Y)):
    if (type(Y[i])==int)==True:
        flow=flow
    elif ((arrta in Y[i])==True) and ((arrtb in Y[i])==True) and ((type(kernel[i])==int)==False):
        flow=flow + kernel[i][ti].integrate_box_1d(stt,end)*M[i][ti][Y[i].get(arrta)][Y[i].get(arrtb)]
    else:
        flow=flow

In [ ]:
flow

# Global model tuned time interval

In [15]:
def transition_matrix(transitions,n):
    n = n #number of states

    M = [[0] * n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def markov_model(record,Y):    
    k=len(record[6].unique())   #number of time intervals
    M=[0] * k                   #transition matrice
    for i in range(k):
        M[i]=transition_matrix(record[record[6]==i][3].map(Y).values,len(Y)) #Transition matrix   
    return M

def distribution(record):
    from scipy import stats
    k=len(record[6].unique())   #number of time intervals
    kernel=[0] * k              #Gaussian KDE kernel
    for i in range(k):
        hs=record[record[6]==i][2].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
        kernel[i]=stats.gaussian_kde(hs)
    return kernel

def local_model_opt(numr_cart):
    record=records_weekday[records_weekday[0]==numr_cart].reset_index(drop=True)  #get record
    
    #record[5]=record[1].apply(lambda x: x.isoweekday()) #seperate weekday/weekend
        
    #record_weekend=record[record[5].isin([6,7])].reset_index(drop=True)
    #record_weekday=record[-record[5].isin([6,7])].reset_index(drop=True)
        
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    X=record[2].apply(lambda x: 
                x.hour * 3600 + x.minute * 60 + x.second).values.reshape(-1,1)
    silhouette=[]
    K=range(2,13)
    for i in K:
        kmeans = KMeans(n_clusters=i, random_state=0).fit(X)
        Z=kmeans.predict(X)
        silhouette.append(silhouette_score(X,Z))
    
    k=np.argmax(silhouette)+2
    kmeans=KMeans(n_clusters=k, random_state=0).fit(X)
    record[6]=kmeans.predict(X)
        
    #train_len=(2*len(record_weekday))//3
    #train=record_weekday[0:train_len]
    #test=record_weekday[train_len:len(record_weekday)].reset_index(drop=True)   #tarin/test set
    
    Y=pd.Series(range(len(record[3].unique())),index=record[3].unique()) #mapping 
    
    M=markov_model(record,Y)
    kernel=distribution(record)
    
    return Y,M,kernel,kmeans

In [ ]:
list_num

In [ ]:
list_num[list_num['3_y']==0]

In [16]:
numk=list_num[(list_num['2_x']>100)&(list_num['1_x']>4)&(list_num['1_x']<5)&(list_num['3_y']==2)&(list_num['3_x']>0)][0].tolist()

In [17]:
len(numk)

6696

In [ ]:
cur.execute('''SELECT DISTINCT numr_cart
            FROM stif_validation_unitaire
            WHERE cod_tlb_arrt_sttn IN ('0030050011000','271007       ')''')
numkk=cur.fetchall()
numkk=[i[0] for i in numkk]

In [ ]:
numr=list(set(numkk).intersection(numk))

In [ ]:
len(numr)

In [18]:
conn = psycopg2.connect(host="192.168.208.49",port="5434",database="iva_db", user="tianru", password="WT:53vh$")
cur=conn.cursor()
start_time=time.time()

numr=numk
Y=[0]*len(numr)
M=[0]*len(numr)
kernel=[0]*len(numr)
kmeans=[0]*len(numr)
for i in range(len(numr)):
    try:
        Y[i],M[i],kernel[i],kmeans[i]=local_model_opt(numr[i])
    except:
        continue
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 2148.5658094882965 seconds ---


In [ ]:
arrta='0030050011000'
arrtb='0030050091300'
for i in range(len(Y)):
    if type(Y[i])==int:
        continue
    elif (arrta in Y[i])==True  and (arrtb in Y[i])==True:
        print(i)
    else:
        continue

In [ ]:
stt=7*60*60
end=7*60*60+30*60
arrta='0030050011000'
arrtb='0030050091300'
flow=0
for i in range(len(Y)):
    if type(Y[i])==int:
        flow=flow
    elif (arrta in Y[i])==True and (arrtb in Y[i])==True:
        ti1=int(kmeans[i].predict(stt))
        ti2=int(kmeans[i].predict(end))
        if ti1==ti2:
            ti=ti1
            flow=flow + (kernel[i][ti].integrate_box_1d(stt,end))*(M[i][ti][Y[i].get(arrta)][Y[i].get(arrtb)])
        else:
            flow=flow
    else:
        flow=flow

In [22]:
cur.execute('''SELECT * FROM stif_arret_station_physique 
            WHERE cod_tlb_arrt_sttn='113001       '
            ''')
cur.fetchall()

[('002',
  2,
  '113001       ',
  '850',
  '800',
  '305',
  '1  ',
  '1',
  '01/01/2006',
  '75110',
  '305',
  '1',
  '01/01/2006',
  'PARIS EST',
  "GARE DE L'EST",
  '1',
  '601682.037',
  '130770.496',
  'O',
  '0',
  '0')]

In [20]:
records_weekday[records_weekday[0].isin(numk)].groupby([3]).count().sort_values([0],ascending=False).reset_index()

,3,0,1,2,5
0,0030050091300,41366,41366,41366,41366
1,271007,34225,34225,34225,34225
2,0030050170401,24187,24187,24187,24187
3,0030050011000,20102,20102,20102,20102
4,281899,19249,19249,19249,19249
5,0030050060600,17908,17908,17908,17908
6,113001,14229,14229,14229,14229
7,0030050170101,13304,13304,13304,13304
8,328328,11287,11287,11287,11287
9,0030030060800,10929,10929,10929,10929


In [19]:
### mse loop
stations=records_weekday[records_weekday[0].isin(numk)].groupby([3]).count().sort_values([0],ascending=False).reset_index()[3]
start_time=time.time()
MSE=[]
for i in range(6):
    for j in range(6):
        flows=[]
        arrta=stations[i]
        arrtb=stations[j]
        for k in range(24):
            try:
                stt=k*60*60
                end=k*60*60+60*60
                flow=0
                for ik in range(len(Y)):
                    if type(Y[ik])==int:
                        flow=flow
                    elif (arrta in Y[ik])==True and (arrtb in Y[ik])==True:
                        ti1=int(kmeans[ik].predict(stt))
                        ti2=int(kmeans[ik].predict(end))
                        if ti1==ti2:
                            ti=ti1
                            flow=flow + (kernel[ik][ti].integrate_box_1d(stt,end))*(M[ik][ti][Y[ik].get(arrta)][Y[ik].get(arrtb)])
                        else:
                            flow=flow
                    else:
                        flow=flow
                flows.append([k,flow])
            except:
                continue
        
        flows_real=[]
        for l in range(24):
            rrr=records_weekday[records_weekday[0].isin(numr)][(records_weekday[2].apply(lambda x: 3600*x.hour + 60*x.minute + x.second)>(k*3600)-1)&(records_weekday[2].apply(lambda x: 3600*x.hour + 60*x.minute + x.second)<((k+1)*3600)+1)].reset_index(drop=True)
            flow=0
            day=[]
            for il in range(len(rrr)-1):
                if rrr[1][il]==rrr[1][il+1] and rrr[3][il]==arrta and rrr[3][il+1]==arrtb:
                    flow=flow+1
                    day.append(rrr[1][il])
                else:
                    flow=flow
            if len(set(day))!=0:
                flows_real.append([l,flow/(len(set(day)))])
            else:
                flows_real.append([l,0])
        
        mse=0
        for m in range(24):
            mse=mse+((flows_real[m][1]-round(flows[m][1]))**2)
        MSE.append(mse/24)
        
print("--- %s seconds ---" % (time.time() - start_time))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


KeyboardInterrupt: 

In [67]:
MSE

[0.95833333333333337,
 0.041666666666666664,
 0.25,
 0.5,
 1.2083333333333333,
 11.875,
 0.70833333333333337,
 0.25,
 0.0,
 0.0,
 0.125,
 1.2083333333333333,
 0.0,
 0.0,
 0.083333333333333329,
 0.91666666666666663,
 0.041666666666666664,
 3.625,
 0.041666666666666664,
 0.0,
 0.125,
 0.0,
 0.0,
 3.5833333333333335,
 0.0,
 0.29166666666666669,
 0.0,
 0.0,
 0.0,
 0.0]

In [69]:
del(MSE[5])

In [59]:
list(MSE[i] for i in [0,3,8,15,24,35])

SyntaxError: invalid syntax (<ipython-input-59-0e02cbf41437>, line 1)

In [70]:
np.mean(MSE)

0.48132183908045972

In [23]:
flows=[]

arrta='271007       '
arrtb='113001       '
for k in range(24):
    try:
        stt=k*60*60
        end=k*60*60+60*60
        flow=0
        for i in range(len(Y)):
            if type(Y[i])==int:
                flow=flow
            elif (arrta in Y[i])==True and (arrtb in Y[i])==True:
                ti1=int(kmeans[i].predict(stt))
                ti2=int(kmeans[i].predict(end))
                if ti1==ti2:
                    ti=ti1
                    flow=flow + (kernel[i][ti].integrate_box_1d(stt,end))*(M[i][ti][Y[i].get(arrta)][Y[i].get(arrtb)])
                else:
                    flow=flow
            else:
                flow=flow
        flows.append([k,flow])
    except:
        continue
flows

[[0, 1.35938445485238e-21],
 [1, 9.1414537288148771e-14],
 [2, 6.778775191538997e-08],
 [3, 0.0005527062508831705],
 [4, 0.062948137451200134],
 [5, 0.39533796622272327],
 [6, 1.1851060821374293],
 [7, 1.5305752579497367],
 [8, 1.2207210162708142],
 [9, 1.0019317551850457],
 [10, 0.34512883538144246],
 [11, 0.86516270384633909],
 [12, 1.4893398815503618],
 [13, 0.4196710393179936],
 [14, 0.41903933221552947],
 [15, 1.8749519388390885],
 [16, 5.2308006742394051],
 [17, 6.989808167852769],
 [18, 3.6554097450979608],
 [19, 1.6485378562696551],
 [20, 0.39231600726421817],
 [21, 0.52895733347466378],
 [22, 0.57894106239882204],
 [23, 0.043554800596249811]]

In [ ]:
flows_real

In [ ]:
### Mean Squared Error
mse=0
for i in range(24):
    mse=mse+((flows_real[i][1]-round(flows[i][1]))**2)
mse/24

In [24]:
flows_real=[]
for k in range(24):
    rrr=records_weekday[records_weekday[0].isin(numr)][(records_weekday[2].apply(lambda x: 3600*x.hour + 60*x.minute + x.second)>(k*3600)-1)&(records_weekday[2].apply(lambda x: 3600*x.hour + 60*x.minute + x.second)<((k+1)*3600)+1)].reset_index(drop=True)
    flow=0
    day=[]
    for i in range(len(rrr)-1):
        if rrr[1][i]==rrr[1][i+1] and rrr[3][i]==arrta and rrr[3][i+1]==arrtb:
            flow=flow+1
            day.append(rrr[1][i])
        else:
            flow=flow
    print(k,flow)
    if len(set(day))!=0:
        flows_real.append([k,flow/(len(set(day)))])
    else:
        flows_real.append([k,0])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


0 0
1 0
2 0
3 0
4 0
5 5
6 3
7 40
8 11
9 2
10 1
11 2
12 8
13 2
14 3
15 21
16 76
17 22
18 13
19 29
20 3
21 5
22 51
23 0


In [ ]:
#cur.close()
#conn.close()